In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, models, datasets
from torch.utils.data import DataLoader, Subset
import numpy as np
from sklearn.metrics import accuracy_score
from torchvision.datasets import SVHN
from torch.optim import lr_scheduler

In [4]:
# Define device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [6]:
# Preprocess the SVHN dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # normalize to range [-1, 1]
])

# Load SVHN dataset
train_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=transform)
test_dataset = datasets.SVHN(root='./data', split='test', download=True, transform=transform)



100%|██████████| 182040794/182040794 [00:05<00:00, 34847749.15it/s]


100%|██████████| 64275384/64275384 [00:01<00:00, 36986800.38it/s]


In [9]:
# Use subset of the dataset (25%)
train_subset_size = int(0.25 * len(train_dataset))
test_subset_size = int(0.25 * len(test_dataset))

train_dataset = torch.utils.data.Subset(train_dataset, np.random.choice(len(train_dataset), train_subset_size, replace=False))
test_dataset = torch.utils.data.Subset(test_dataset, np.random.choice(len(test_dataset), test_subset_size, replace=False))


In [10]:
# Define dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [12]:
import torch
import torch.nn as nn
import torchvision.models as models

# Function to modify the last layer of pretrained models for SVHN
def modify_last_layer_for_svhn(model):
    if isinstance(model, models.AlexNet):
        num_ftrs = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_ftrs, 10)  # Assuming 10 classes in SVHN dataset
    elif isinstance(model, models.VGG):
        num_ftrs = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_ftrs, 10)
    elif isinstance(model, models.ResNet):
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, 10)
    # elif isinstance(model, models.LeNet):
    #     num_ftrs = model.fc3.in_features
    #     model.fc3 = nn.Linear(num_ftrs, 10)
    else:
        raise ValueError("Unsupported model architecture")

# Load various pretrained models
models_dict = {
    # "LeNet-5": models.LeNet(),
    "AlexNet": models.alexnet(pretrained=True),
    "VGG-16": models.vgg16(pretrained=True),
    "ResNet-18": models.resnet18(pretrained=True),
    "ResNet-50": models.resnet50(pretrained=True),
    "ResNet-101": models.resnet101(pretrained=True)
}

# Modify the last layer for SVHN dataset
for model_name, model in models_dict.items():
    modify_last_layer_for_svhn(model)
    print(f"Modified {model_name} for SVHN dataset:\n{model}\n")


Modified AlexNet for SVHN dataset:
AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=92

In [14]:
# Train and evaluate models
for model_name, model in models_dict.items():
    print(f"Training {model_name}...")
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    for epoch in range(3):  # Train for 5 epochs
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        scheduler.step()
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{3}, Loss: {epoch_loss:.4f}")

    # Evaluate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f"Accuracy of {model_name}: {accuracy:.4f}")

Training AlexNet...
Epoch 1/3, Loss: 0.5230
Epoch 2/3, Loss: 0.4016
Epoch 3/3, Loss: 0.3401
Accuracy of AlexNet: 0.9104
Training VGG-16...
Epoch 1/3, Loss: 0.8878
Epoch 2/3, Loss: 0.3867
Epoch 3/3, Loss: 0.3070
Accuracy of VGG-16: 0.9096
Training ResNet-18...
Epoch 1/3, Loss: 0.9027
Epoch 2/3, Loss: 0.2912
Epoch 3/3, Loss: 0.2079
Accuracy of ResNet-18: 0.9341
Training ResNet-50...
Epoch 1/3, Loss: 0.9582
Epoch 2/3, Loss: 0.2799
Epoch 3/3, Loss: 0.1798
Accuracy of ResNet-50: 0.9427
Training ResNet-101...
Epoch 1/3, Loss: 0.8702
Epoch 2/3, Loss: 0.2385
Epoch 3/3, Loss: 0.1429
Accuracy of ResNet-101: 0.9448
